In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_openai.embeddings import OpenAIEmbeddings
from supabase import create_client
import json
import os

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")


In [2]:
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

In [3]:
def get_book(query):
    query_embedding = OpenAIEmbeddings(model="text-embedding-3-small").embed_query(query)

    response = supabase.rpc("match_book", {
        "query_embedding" : query_embedding, "filter": json.dumps({})
        }).execute().data

    context_texts = "\n\n".join([
        f"[{doc.get('name', '제목 없음')}]\n{doc['description']}" for doc in response
    ])

    return context_texts

In [4]:
from langchain.prompts import ChatPromptTemplate
question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
         You are a professional book curator who recommends books by analyzing their summaries.
        You understand the reader's intent from their search query, and you provide thoughtful recommendations with clear reasoning.

        Below is the user's search query and a list of books whose summaries were retrieved based on that query.

        📌 User's query:
        "{query}"

        📚 Relevant book summaries:
        {context}

        Please recommend books from the context that best match the user's query.
        For each book, explain *why* it fits the query.

        Use the tone, format, and detail level of the following example recommendation as a model:

        ✨ Sample recommendation to imitate:
        {book_recommendation}

        Please write in Korean.
        Now, generate your recommendation below in the same style:
        """),
    ]
)

In [19]:
from langchain_core.runnables import RunnablePassthrough

query = "철학"
chain = RunnablePassthrough() | question_prompt | llm

result = chain.invoke({"context" : get_book(query), "query" : query, "book_recommendation" : "None"})

In [17]:
result.content

'사용자의 검색 쿼리 "구의 증명"과 관련하여 적합한 책을 추천드립니다.\n\n1. **《오로라》 - 최진영**\n   이 책은 최진영 작가의 신작 소설로, 제주를 배경으로 한 사랑 이야기를 다룹니다. "구의 증명"이라는 제목이 주는 수학적 또는 철학적 의미와는 다소 거리가 있지만, 사랑의 복잡성과 인간의 내면을 탐구하는 내용이 독자의 호기심을 자극할 수 있습니다. 특히, 주인공이 자유를 찾고 새로운 정체성을 형성하는 과정은 "구의 증명"이 갖는 논리적이고 체계적인 사고와 연결될 수 있습니다. 또한, 제주라는 장소가 주는 독특한 분위기와 감정의 변화는 독자가 새로운 시각에서 사랑을 바라보게 할 것입니다.\n\n이 외의 책들은 주제나 내용이 "구의 증명"과 직접적인 관련이 없거나, 수학적 사고와는 거리가 있어 추천하지 않았습니다. "오로라"는 독자의 감정과 사고를 자극할 수 있는 요소를 지니고 있어, 흥미로운 독서 경험을 제공할 것입니다.'